In [1]:
import requests
from pydantic import BaseModel, ConfigDict, Field, field_validator, ValidationError, IPvAnyAddress

In [2]:
class IPGeo(BaseModel):
    model_config = ConfigDict(extra="ignore")

    ip: IPvAnyAddress
    country: str | None = None
    country_code: str | None = Field(default=None, min_length=2, max_length=2)
    country_code3: str | None = Field(default=None, min_length=3, max_length=3)
    city: str | None = None
    region: str | None = None
    timezone: str | None = None
    organization_name: str | None = None

    @field_validator("organization_name", mode="after")
    @classmethod
    def set_unknown_to_none(cls, v: str) -> str:
        if v.casefold() == "unknown":
            return None
        return v



In [8]:
class MyIPResponse(BaseModel):
    ip: IPvAnyAddress
    country: str | None = None
    cc: str | None = None

In [12]:
def get_my_api():
    my_api_response = requests.get("https://api.myip.com/")
    return MyIPResponse.model_validate(my_api_response.json())

In [13]:
url_query = "https://get.geojs.io/v1/ip/geo/{ip_address}.json"

In [14]:
url = url_query.format(ip_address=get_my_api().ip)

In [15]:
response = requests.get(url)
response.raise_for_status()

IPGeo.model_validate(response.json())

IPGeo(ip=IPv4Address('188.169.72.113'), country='Georgia', country_code='GE', country_code3='GEO', city='Tbilisi', region='Tbilisi', timezone='Asia/Tbilisi', organization_name='JSC Silknet')